In [1]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from app.environment.dataprovider import DataProvider
from app.preparation.preparator import DataPreparator

In [2]:
def is_increasing(window):
    return ((window[0][-3] > window[0][-2] and window[0][-2] < window[0][-1]) and
                (window[1][-3] > window[1][-2] and window[1][-2] < window[1][-1]) and
                (window[2][-3] > window[2][-2] and window[2][-2] < window[2][-1]) and
                (window[3][-3] > window[3][-2] and window[3][-2] < window[3][-1]))

def is_decreasing(window):
    return ((window[0][-3] < window[0][-2] and window[0][-2] > window[0][-1]) and
                (window[1][-3] < window[1][-2] and window[1][-2] > window[1][-1]) and
                (window[2][-3] < window[2][-2] and window[2][-2] > window[2][-1]) and
                (window[3][-3] < window[3][-2] and window[3][-2] > window[3][-1]))


In [7]:
days = 5
start_date = '2000-01-01'
end_date = '2021-12-31'
start_capital = 100_000.0
order_fee_in_dollar = 1.0
transaction_fee_in_percent = 0.0
spread_in_percent = 0.5
tax_in_percent = 25.0
solidarity_tax_in_percent = 5.5

api_key = os.getenv('TIINGO_API_KEY')
provider = DataProvider(api_key)
for ticker, company in provider.tickers.items():
    data = provider.load(ticker, start_date, end_date, enable_cached_data=True)
    data['normalized'] = \
                        DataPreparator.calculate_windows(
                            data,
                            days=days,
                            normalize=True,
                            columns=['open', 'high', 'low', 'close'],
                            adjust=DataProvider.adjust_prices,
                            smooth=True)
    data = data[['date', 'close', 'adj_close', 'normalized']].copy()
    capital = start_capital
    stock_count = 0
    buy_price = 0.0
    sell_price = 0.0
    buy_transaction = 0.0
    sell_transaction = 0.0
    tax_rate = (tax_in_percent / 100.0) * ((100.0 + solidarity_tax_in_percent) / 100.0)
    for i, row in data.iterrows():
        window = row['normalized'].reshape(4, days)
        if stock_count == 0 and is_increasing(window):
            buy_price = row['adj_close']
            stock_count = int(capital / buy_price)
            buy_transaction = stock_count * buy_price
            capital -= order_fee_in_dollar
            capital -= buy_transaction * (1.0 + (transaction_fee_in_percent / 100.0))
        elif stock_count > 0 and is_decreasing(window):
            sell_price = row['adj_close']
            sell_transaction = stock_count * sell_price
            earnings = sell_transaction - buy_transaction
            capital += sell_transaction
            capital -= order_fee_in_dollar
            capital -= sell_transaction * (transaction_fee_in_percent / 100.0)
            if earnings > 0.0:
                tax = tax_rate * earnings
                capital -= tax
            stock_count = 0
            buy_price = 0.0
            sell_price = 0.0
            buy_transaction = 0.0
            sell_transaction = 0.0
    print(f'{ticker} - {company} - Total: {capital + buy_transaction:.2f}')

MMM - 3M Company - Total: 123053.93
ABBV - AbbVie Inc - Total: 236247.90
ABMD - ABIOMED Inc - Total: 121205.32
ATVI - Activision Blizzard - Total: 3939301.90
ADBE - Adobe Systems Inc. - Total: 143896.65
AMD - Advanced Micro Devices Inc - Total: 144110.88
A - Agilent Technologies Inc - Total: 32039.20
AGYS - Agilysys Inc - Total: 175634.04
ALB - Albemarle Corp - Total: 84475.86
GOOGL - Alphabet Inc Class A - Total: 700094.86
AMZN - Amazon Inc. - Total: 996883.65
AAL - American Airlines Group Inc - Total: 43512.30
AMGN - Amgen Inc. - Total: 127256.53
AXP - American Express Company AMEX - Total: 54939.82
AAPL - Apple Inc. - Total: 443108.40
T - AT & T - Total: 72090.43
BIDU - Baidu Inc. - Total: 216417.81
BAC - Bank of America Corp - Total: 127428.29
GOLD - Barrick Gold Corporation BC - Total: 228517.97
BIIB - Biogen Idec Inc - Total: 286023.95
BB - BlackBerry Ltd - Total: 675900.75
BA - Boeing Company - Total: 196413.03
BKNG - Booking Holdings Inc - Total: 761512.51
BXP - Boston Properti

KeyboardInterrupt: 